In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1 par
PosNum = 300
NegNum = 500
winSize = (40, 40)
blockSize = (16, 16)  # 105
blockStride = (8, 8)  # 4 cell
cellSize = (8, 8)
nBin = 9  # 9 bin 


In [8]:
# 2 hog create hog 1 win 2 block 3 blockStride 4 cell 5 bin
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nBin)

# 3 svm
svm = cv2.ml.SVM_create()

# 4 computer hog
featureNum = int(((40 - 16) / 8 + 1) * ((40 - 16) / 8 + 1) * 4 * 9)  # 576
featureArray = np.zeros(((PosNum+NegNum),featureNum),np.float32)
labelArray = np.zeros(((PosNum+NegNum),1),np.int32)

for i in range(0, PosNum):
    fileName = 'data/positive/' + str(i) + '.bmp'
    img = cv2.imread(fileName)
    hist = hog.compute(img, (8,8))  # 576
    featureArray[i: ] =  hist.reshape(576)
    labelArray[i, 0] = 1

for i in range(0, NegNum):
    fileName = 'data/negative/' + str(i) + '.bmp'
    img = cv2.imread(fileName)
    hist = hog.compute(img, (8,8))  # 576
    featureArray[i + PosNum: ] = hist.reshape(576)
    labelArray[i + PosNum, 0] = -1

In [10]:
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
# 6 train
ret = svm.train(featureArray,cv2.ml.ROW_SAMPLE,labelArray)

svm.save('opencv_svm_bird.xml')

In [29]:
# 检测
alpha = np.zeros((1),np.float32)
rho = svm.getDecisionFunction(0,alpha)
alphaArray = np.zeros((1,1),np.float32)
supportVArray = np.zeros((1,featureNum),np.float32)
resultArray = np.zeros((1,featureNum),np.float32)
alphaArray[0,0] = alpha
resultArray = -1*alphaArray*supportVArray
# detect
myDetect = np.zeros((577),np.float32)
for i in range(0,576):
    myDetect[i] = resultArray[0,i]
myDetect[576] = rho[0]
# rho svm （判决）
myHog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nBin)
myHog.setSVMDetector(myDetect)
# myHog.load('opencv_svm_bird.xml')
# load 
imageSrc = cv2.imread('data/test/red_one.png',1)
print(imageSrc.shape)
# (8,8) win 
# error: dsize.area() > 0 || (inv_scale_x > 0 && inv_scale_y > 0) in function cv::resize
# https://github.com/msracver/FCIS/issues/113
rects, objs = myHog.detectMultiScale(imageSrc,0,(8,8),(50,50),1.05,2)
print(rects)
# xy wh 三维 最后一维
for (x, y, w, h) in rects:
    cv2.rectangle(imageSrc, (x, y), (x + w, y + h), (0, 0, 255), 2)
cv2.imshow('dst',imageSrc)
cv2.waitKey(0)

(772, 1535, 3)
()


-1